In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text

In [4]:
iris = load_iris()

X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree Classifier: {accuracy:.2f}")

Accuracy of the Decision Tree Classifier: 1.00


In [7]:
print('Tree rules: ')
print(export_text(clf, feature_names=iris.feature_names))

Tree rules: 
|--- petal length (cm) <= 2.45
|   |--- class: 0
|--- petal length (cm) >  2.45
|   |--- petal length (cm) <= 4.75
|   |   |--- petal width (cm) <= 1.65
|   |   |   |--- class: 1
|   |   |--- petal width (cm) >  1.65
|   |   |   |--- class: 2
|   |--- petal length (cm) >  4.75
|   |   |--- petal width (cm) <= 1.75
|   |   |   |--- petal length (cm) <= 4.95
|   |   |   |   |--- class: 1
|   |   |   |--- petal length (cm) >  4.95
|   |   |   |   |--- petal width (cm) <= 1.55
|   |   |   |   |   |--- class: 2
|   |   |   |   |--- petal width (cm) >  1.55
|   |   |   |   |   |--- petal length (cm) <= 5.45
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- petal length (cm) >  5.45
|   |   |   |   |   |   |--- class: 2
|   |   |--- petal width (cm) >  1.75
|   |   |   |--- petal length (cm) <= 4.85
|   |   |   |   |--- sepal width (cm) <= 3.10
|   |   |   |   |   |--- class: 2
|   |   |   |   |--- sepal width (cm) >  3.10
|   |   |   |   |   |--- class: 1
|   |   | 

In [21]:
from sklearn.tree import DecisionTreeClassifier

def tree_to_lisp(tree, feature_names=None, node=0):
    """
    Wandelt einen sklearn-Baum in eine Lisp-S-Expression um.
    """
    left = tree.children_left[node]
    right = tree.children_right[node]
    if feature_names is None:
        feature = f"X{tree.feature[node]}"
    else:
        feature = feature_names[tree.feature[node]]
    threshold = tree.threshold[node]
    
    # Blattknoten?
    if left == -1 and right == -1:
        value = tree.value[node].tolist()
        return str(value)
    
    # Rekursiver Fall: (if (> feature threshold) then else)
    then_branch = tree_to_lisp(tree, feature_names, left)
    else_branch = tree_to_lisp(tree, feature_names, right)
    return ('if', ('>', feature, threshold), then_branch, else_branch)

# Beispielnutzung:
clf = DecisionTreeClassifier().fit(X_train, y_train)
lisp_repr = tree_to_lisp(clf.tree_, 
                         feature_names=iris.feature_names
                         )
print(lisp_repr)


('if', ('>', 'petal length (cm)', 2.449999988079071), '[[40.0, 0.0, 0.0]]', ('if', ('>', 'petal length (cm)', 4.75), ('if', ('>', 'petal width (cm)', 1.6500000357627869), '[[0.0, 36.0, 0.0]]', '[[0.0, 0.0, 1.0]]'), ('if', ('>', 'petal width (cm)', 1.75), ('if', ('>', 'petal length (cm)', 4.950000047683716), '[[0.0, 2.0, 0.0]]', ('if', ('>', 'petal width (cm)', 1.550000011920929), '[[0.0, 0.0, 3.0]]', ('if', ('>', 'petal length (cm)', 5.450000047683716), '[[0.0, 2.0, 0.0]]', '[[0.0, 0.0, 1.0]]'))), ('if', ('>', 'petal length (cm)', 4.8500001430511475), ('if', ('>', 'sepal length (cm)', 5.950000047683716), '[[0.0, 1.0, 0.0]]', '[[0.0, 0.0, 2.0]]'), '[[0.0, 0.0, 32.0]]'))))


In [27]:
clf = DecisionTreeClassifier().fit(X_train, y_train)
feature_names = [f"X{i}" for i in range(X_train.shape[1])]

In [44]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

def tree_to_lisp(clf, feature_names=None, node=0):
    """
    Wandelt einen sklearn-Baum in eine Lisp-S-Expression um,
    wobei jedes Blatt die vorhergesagte Klasse zurückgibt.
    """
    tree = clf.tree_
    left, right = tree.children_left[node], tree.children_right[node]
    
    if left == -1 and right == -1:
        vals = tree.value[node][0]
        idx = int(np.argmax(vals))
        klass = clf.classes_[idx]
        return repr(klass)
    
    if feature_names is None:
        feat = f"X{tree.feature[node]}"
    else:
        feat = feature_names[tree.feature[node]]
    thresh = tree.threshold[node]
    
    then_branch = tree_to_lisp(clf, feature_names, left)
    else_branch = tree_to_lisp(clf, feature_names, right)
    return ((feat, thresh), then_branch, else_branch)

# Beispielaufruf:
print(tree_to_lisp(clf, feature_names))

(('X3', 0.800000011920929), '0', (('X2', 4.75), (('X3', 1.6500000357627869), '1', '2'), (('X3', 1.75), (('X2', 4.950000047683716), '1', (('X3', 1.550000011920929), '2', (('X0', 6.949999809265137), '1', '2'))), (('X2', 4.8500001430511475), (('X1', 3.100000023841858), '2', '1'), '2'))))
